In [2]:
# 대한민국 구석구석 사이트 상세 정보 수집하기

#Step 1. 필요한 모듈을 로딩합니다
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import time          
import pandas as pd    
import math

#Step 2. 사용자에게 검색 관련 정보들을 입력 받습니다.
print("=" *100)
print(" 연습문제: 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.")
print("=" *100)

query_txt = input('1.정보를 수집할 키워드는 무엇입니까?: ')

cnt = int(input('2.몇 건의 정보를 수집할까요? :'))
page_cnt = math.ceil( cnt / 10 )
print('총 %s 건의 정보를 수집하기 위해 %s 페이지까지 이동할 예정입니다' %(cnt , page_cnt))

#Step 3. 수집된 데이터를 저장할 폴더 이름 입력받기 
f_dir = input("3.파일을 저장할 폴더명만 쓰세요(기본값:c:\\py_temp\\):")
if f_dir == '' :
    f_dir="c:\\py_temp\\"
    
print("\n")    

#Step 3. 검색어 입력한 후 검색하여 해당 장르로 이동하기
s = Service("c:/py_temp/chromedriver.exe")
driver = webdriver.Chrome(service=s)

query_url = 'https://korean.visitkorea.or.kr/'

driver.get(query_url)
time.sleep(2)
driver.maximize_window()

s_time = time.time( )
element = driver.find_element(By.ID,"inp_search")
element.send_keys(query_txt)
element.send_keys("\n")
time.sleep(5)

# Step 5. 본문 내역 추출하기
time.sleep(0.2)
s_time = time.time( ) 

no2=[]           # 게시글 번호 컬럼
title2=[ ]       # 게시글 제목 컬럼
org2=[]          # 지자체명 컬럼
contents2=[ ]     # 본문 내용 컬럼
tag2=[ ]         # 해시태그 컬럼

no = 1           # 게시글 번호 초기값

for a in range(1,page_cnt+1) :
    print('')
    print('현재 %s 페이지에 있는 정보를 수집합니다' %a)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    content_list = soup.find('ul','list_thumType type1').find_all('li')
    cnt_no = 1
    
    for b in content_list:
        
        try :
            title = b.find('div','area_txt').find('div','tit').get_text().strip()
        except :
            continue
        else :
            # 1. 게시글 번호
            print("\n")
            print("%s 번째 정보를 추출하고 있습니다============" %no)
            no2.append(no)
            print("1.번호 : %s" %no)

            # 2. 게시글 제목
            title2.append(title)
            print("2.제목 : %s" %title )
            
            # 3. 지자체이름
            org = b.find('div','service').find('p').get_text().strip()
            org2.append(org)
            print("3.지자체이름: %s" %org)
            
            #4. 해시태그
            try :
                tag = b.find('p','tag_type').get_text().replace("#"," #")
            except :
                tag = ' '
                
            tag2.append(tag)
            print("4.해시태그: %s" %tag)
            
            #5. 본문   
            # 1 페이지에만 광고가 들어 있어서 xpath 값이 다름
            # 2 페이지부터는 광고가 없어서 1씩 증가시키면 됨
            if a == 1 and cnt_no == 5 :
                cnt_no += 1
                
            driver.find_element(By.XPATH,'//*[@id="listBody"]/ul/li[%s]/div[2]/div[1]/a' %cnt_no).click()
            cnt_no += 1
            
            time.sleep(5)
            
            html_2 = driver.page_source
            soup_2 = BeautifulSoup(html_2, 'html.parser')
            
            content1 = soup_2.find('div','inr_wrap')
            content2 = soup_2.find('div','box_txtPhoto')
            
            if content1 :
                content = content1.find('div','inr').find('p').get_text()
                contents2.append(content)
                print('5.본문내용:', content)
            elif content2 :
                content_1 = soup_2.find('div','box_txtPhoto')
                try :
                    content_1.style.decompose()
                    content_1.script.decompose()
                except :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
                else :
                    content_2 = content_1.find_all('div','txt_p')
                    print('5.본문내용:') 
                    for b in content_2 :
                        content = b.get_text().strip()
                        print(content)
                        contents2.append(content)
                           
            driver.back()
            
            time.sleep(5)
                       
            no += 1         # 전체 게시글 번호용 값
            
            if no > cnt :
                 break
                
            time.sleep(0.5)
            
    a += 1
    try :
        driver.find_element(By.LINK_TEXT,'%s' %a).click()
    except :
        driver.find_element(By.LINK_TEXT,'다음').click()

    time.sleep(0.5)

# Step 6. 결과 저장하기
import time
import os

n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' %(n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)

os.makedirs(f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt)

fc_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.csv'
fx_name = f_dir+'대한민국구석구석'+'-'+s+'-'+query_txt+'\\'+'대한민국구석구석'+'-'+s+'-'+query_txt+'.xls'

df = pd.DataFrame()
df['번호']=no2
df['제목']=pd.Series(title2)
df['지자체명']=pd.Series(org2)
df['본문내용']=pd.Series(contents2)
df['해시태그']=pd.Series(tag2)

# xls , csv 형태로 저장하기
df.to_excel(fx_name,index=False, encoding="utf-8", engine='openpyxl')
df.to_csv(fc_name,index=False, encoding="utf-8-sig")

e_time = time.time( )     
t_time = e_time - s_time

print("\n") 
print("=" *80)
print("크롤링을 요청한 총 %s 건 중에서 %s 건의 데이터를 수집 완료 했습니다" %(cnt,no-1))
print('xls 저장경로: ',fx_name)
print('csv 저장경로: ',fc_name)
print("총 소요시간은 %s 초 입니다 " %round(t_time,1))
print("=" *80)

driver.close( )

 연습문제: 이 크롤러는 대한민국 구석구석 사이트 정보 수집용 웹크롤러입니다.
1.정보를 수집할 키워드는 무엇입니까?: 제주도
2.몇 건의 정보를 수집할까요? :35
총 35 건의 정보를 수집하기 위해 4 페이지까지 이동할 예정입니다
3.파일을 저장할 폴더명만 쓰세요(기본값:c:\py_temp\):



현재 1 페이지에 있는 정보를 수집합니다


1 번째 정보를 추출하고 있습니다============
1.번호 : 1
2.제목 : 겨울에 만나는 초록빛 곶자왈, 제주 청수마을
3.지자체이름: 제주도 제주시
4.해시태그:  #2022년2월추천가볼만한곳 #2월추천가볼만한곳 #추천가볼만한곳 #제주가볼만한곳 #청수마을 #예술곶 #용수항 #성김대건신부제주표착기념관 #자구내포구 #차귀도 #수월봉 #공공누리 #겨울여행 #제주도 


WebDriverException: Message: unknown error: unexpected command response
  (Session info: chrome=103.0.5060.114)
Stacktrace:
Backtrace:
	Ordinal0 [0x0065D953+2414931]
	Ordinal0 [0x005EF5E1+1963489]
	Ordinal0 [0x004DC6B8+837304]
	Ordinal0 [0x004CEB34+781108]
	Ordinal0 [0x004CE06A+778346]
	Ordinal0 [0x004CD646+775750]
	Ordinal0 [0x004CC565+771429]
	Ordinal0 [0x004CCB68+772968]
	Ordinal0 [0x004CCAF8+772856]
	Ordinal0 [0x004D2D5A+798042]
	Ordinal0 [0x004CDD3B+777531]
	Ordinal0 [0x004CE265+778853]
	Ordinal0 [0x004CE04F+778319]
	Ordinal0 [0x004CD646+775750]
	Ordinal0 [0x004CC565+771429]
	Ordinal0 [0x004CCB68+772968]
	Ordinal0 [0x004CCAF8+772856]
	Ordinal0 [0x004D27F6+796662]
	Ordinal0 [0x004D2B2A+797482]
	Ordinal0 [0x004CDD3B+777531]
	Ordinal0 [0x004CE265+778853]
	Ordinal0 [0x004CE04F+778319]
	Ordinal0 [0x004CD646+775750]
	Ordinal0 [0x004CC565+771429]
	Ordinal0 [0x004CCB68+772968]
	Ordinal0 [0x004CCAF8+772856]
	Ordinal0 [0x004D27F6+796662]
	Ordinal0 [0x004D2B2A+797482]
	Ordinal0 [0x004CDD3B+7

In [17]:
print(cnt_no)

11
